# 导入必要的包

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
import pandas as pd # 读取数据
from torch import nn

# 读取数据集

In [3]:
true_w1 = torch.tensor([2, -3.4, 5.6])
true_w2 = torch.tensor([4.2, 2.1, -3.1])
true_b1 = 4.2
true_b2 = 4.2

# 生成数据集
# 3 个特征，2 个标签
# 1000 个样本
in_features = 3
out_features = 2
num_examples = 1000


# 生成特征矩阵 X，正态分布随机值，形状为 (num_examples, n_features)
features = torch.normal(0, 1, (num_examples, in_features))
# features 是一个 (num_examples, in_features) 的矩阵
y1 = torch.matmul(features, true_w1) + true_b1
y2 = torch.matmul(features, true_w2) + true_b2
# 添加随机噪声，模拟真实数据中的误差
y1 += torch.normal(0, 0.01, y1.shape)
y2 += torch.normal(0, 0.01, y2.shape)

labels = torch.stack((y1, y2), 1)
# labels 是一个 (num_examples, out_features) 的矩阵

# 读取并加载数据


In [4]:
def load_array(data_arrays, batch_size, is_train=True):
    """
    构造一个 PyTorch 数据迭代器。

    根据给定的数据数组创建一个 `DataLoader`，用于批量加载数据，支持随机打乱数据顺序。

    参数:
        data_arrays (tuple[torch.Tensor, ...]): 包含特征和标签的元组，每个元素是一个 `torch.Tensor`。
        batch_size (int): 每个小批量的样本数量。
        is_train (bool): 是否将数据打乱（`True` 表示训练模式，数据会随机打乱；`False` 表示测试模式，数据按顺序加载）。

    返回:
        torch.utils.data.DataLoader: 一个数据加载器对象，用于按批量加载数据。
    """
    # 使用 TensorDataset 将特征和标签封装成一个数据集对象
    dataset = data.TensorDataset(*data_arrays)

    # 创建 DataLoader，指定批量大小和是否随机打乱数据
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

# 检验迭代器是否正常工作

In [5]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-0.3469, -0.5399, -0.5998],
         [-0.9775, -0.8029,  0.0220],
         [-1.2920,  1.0517,  0.2393],
         [ 0.8075,  0.4184, -0.5507],
         [ 2.1325,  0.8381, -0.9595],
         [ 0.4157, -0.7942,  1.2377],
         [ 0.4274,  0.6164, -1.3207],
         [-0.3655, -0.7261, -0.7089],
         [ 0.6097, -1.4632,  0.6271],
         [-0.8005,  0.2740,  1.0022]]),
 tensor([[ 1.9755,  3.4669],
         [ 5.0889, -1.6675],
         [-0.6047,  0.2288],
         [ 1.2882, 10.1939],
         [ 0.2430, 17.8943],
         [14.6527,  0.4284],
         [-4.4460, 11.3782],
         [ 1.9579,  3.3265],
         [13.9044,  1.7502],
         [ 7.2685, -1.6833]])]

# 定义线性模型

In [6]:
in_features = 3
out_features = 2
net = nn.Sequential(nn.Linear(in_features, out_features))
# nn是神经网络的缩写
# Sequential是一个有序容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中执行
# Linear是一个全连接层，它接收一个输入张量，然后计算输出

# 初始化模型参数

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0., 0.])

# 定义损失函数

In [8]:
loss = nn.MSELoss()

# 定义优化算法

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
# parameters()返回一个包含模型所有参数的迭代器

# 训练

In [12]:
num_epochs = 3  # 设置训练的总轮数
for epoch in range(num_epochs):  # 遍历每一轮训练
    for X, y in data_iter:  # 遍历每个小批量数据
        # 计算当前批量数据的损失
        l = loss(net(X), y)

        # 梯度清零，防止累积
        trainer.zero_grad()  

        # 计算梯度
        l.backward()  

        # 根据梯度更新参数
        trainer.step()  

    # 在每轮训练结束后，计算整个数据集上的损失
    l = loss(net(features), labels)  

    # 打印当前轮次和对应的损失
    print(f'epoch {epoch + 1}, loss {l:f}')


epoch 1, loss 0.000095
epoch 2, loss 0.000095
epoch 3, loss 0.000095


# 最后比较一下训练的结果

In [13]:
w = net[0].weight.data
# w 是一个形状为 (out_features, in_features) 的张量
print('w1的估计误差：', true_w1 - w[0].reshape(in_features))
print('w2的估计误差：', true_w2 - w[1].reshape(in_features))
b = net[0].bias.data
print('b的估计误差：', true_b1 - b[0])
print('b的估计误差：', true_b2 - b[1])

w1的估计误差： tensor([0.0008, 0.0003, 0.0002])
w2的估计误差： tensor([-3.5286e-05,  2.5439e-04,  2.0528e-04])
b的估计误差： tensor(2.0504e-05)
b的估计误差： tensor(-0.0004)
